In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print('gpu', gpu)
    tf.config.experimental.set_memory_growth(gpu, True)
    print('memory growth:' , tf.config.experimental.get_memory_growth(gpu))


gpu PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
memory growth: True


### Download and parse dataset

In [2]:
import numpy as np

path = tf.keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt'
)
text = open(path).read().lower()
print(f'Corpus len: {len(text)}')

606208/600901 [==============================] - 1s 1us/step
Corpus len: 600893


### Write sequences of characters as vectors

In [4]:
maxlen=60 #sequence of 60 characters
step=3 # new sequence every 3 characters
sentences=[] #store sequences
next_chars=[] #store next characters

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i:i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences: ', len(sentences))

#unique characters
chars = sorted(list(set(text)))
print('Number of unique characters', len(chars))
char_indices = dict((char, char.index(char)) for char in chars)

print('Creating vectors...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences:  200278
Number of unique characters 57
Creating vectors...


### Model

In [6]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(tf.keras.layers.Dense(len(chars), activation="softmax"))

In [7]:
optimizer = tf.keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [8]:
def sample(preds, temperature=1.0):
    preds=np.asarray(preds).astype('float64')
    preds=np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)